In [1]:
import re
import os
import glob


def parsetop(inputfile):
    # newpath = os.path.abspath ( '')+'/'+str(outputfile)+'/'    # input file
    # os.makedirs(newpath,exist_ok=True)
    with open(inputfile, "r") as fp:
        original_lines = fp.readlines()

    lines = [line for line in original_lines if line.strip()]
    number = []
    includelines_number = []
    lineNumber = 1
    lineNumber_include = 1
    keyword1 = "]"  # input('Keyword:')
    for eachline in lines:  # search for keywords and get linenumber
        m = re.search(keyword1, eachline)
        if m is not None:
            number.append(lineNumber - 1)  # split by linenumber
        lineNumber += 1

    keyword2 = "#include"  # input('Keyword:')
    for eachline in lines:  # search for keywords and get linenumber
        n = re.search(keyword2, eachline)
        if n is not None:
            includelines_number.append(lineNumber_include - 1)  # split by linenumber
        lineNumber_include += 1
    includelines_number.sort()
    number.append(includelines_number[0])

    number.append(len(lines))
    number = list(set(number))
    number.sort()
    size = int(len(number))
    # print(number)

    middlelines = []
    sectorname = []
    for i in range(size - 1):
        # set output range
        start = number[i]
        end = number[i + 1]
        middlelines.append(lines[start:end])
        sectorname.append(lines[start])

        # fp_w = open(sectorname+'.txt','w')
        # for key in middlelines:
        #    fp_w.write(key)
        # fp_w.close()
    return middlelines, sectorname

In [2]:
middlelines,sectorname=parsetop('template.top')


In [ ]:
#grofile_name = "TCPP_lacvpss"
#itp_files = glob.glob(
#    os.path.join("**/" + "new" + grofile_name + "*.itp"), recursive=True
#)
#subfolders = [os.path.dirname(itpfile) for itpfile in itp_files]
#
#subfolders, itp_files

In [18]:
# fetch atomtype sector
#for i in range(len(subfolders)):
def top_gen(itp_file):
    with open(itp_file, "r") as f:
        lines = f.readlines()

    keyword1 = "atomtype"
    keyword2 = "moleculetype"
    for eachline in lines:  # search for keywords and get linenumber
        if re.search(keyword1, eachline):
            start = lines.index(eachline) + 2
        elif re.search(keyword2, eachline):
            end = lines.index(eachline) - 1

    target_lines = [line for line in lines[start:end] if line.strip()]

    newstart = end + 1

    with open("../itps/linker.itp", "w") as fp:
        fp.writelines(lines[newstart:])

    target_lines.append("\n")
    sec1 = target_lines
    sec2 = [
        '#include "../itps/linker.itp"'
        + "\n"
        + '#include "../itp/tip3p.itp"'
        #+ "\n"
        #+ '#include "../itp/K.itp"'
        + "\n"
        + "\n"
    ]
    sec3 = ["MOF_UIO66" + "\n" + "\n"]
    sec4 = [

        "\nZRR            48\nOMM            32\nOH            32\nCUT             60\n"
        + "UNK            18\nSOL            1000\n"
        + "\n"
        + "\n"
    ]
    newtop = (
        middlelines[0]
        + middlelines[1]
        + sec1
        + middlelines[2]
        + sec2
        + middlelines[3]
        + sec3
        + middlelines[4]
        + sec4
    )
    with open("em.top", "w") as f:
        f.writelines(newtop)

In [19]:
top_gen('../itps/cph.itp')

In [ ]:
#grofile_name = 'TCPP_lacvpss' 
#for itpfolder in subfolders:
#    ss_files = glob.glob(os.path.join(itpfolder+'/*ss.*'), recursive=True)
#    file_names = [os.path.basename(file) for file in ss_files]
#    file_names.append("all.xyz")
#    file_names.append("all.gro")
#    os.makedirs(itpfolder+'/otherfiles',exist_ok=True)
#    for eachfile in file_names:
#        os.rename(itpfolder+'/'+eachfile,itpfolder+'/otherfiles/'+eachfile)


In [ ]:
#neutral -4TCP
for index in neu_box[2:3]:
    a = subfolders[index]
    top_gen(itp_files,subfolders,index,0)
    !gmx insert-molecules -f $a/newall.gro -ci itp/tip3p.gro -nmol 3400 -o $a/em.gro   
    #!gmx insert-molecules -f $a/mof_sol.gro -ci itp/K.gro -nmol 10 -o $a/mof_sol_K.gro  
    !gmx grompp -f mdp/em.mdp   -c $a/em.gro    -p $a/em.top   -o $a/em.tpr -maxwarn 1 

In [ ]:
#neg-1 -5TCP
for index in n1_box:
    a = subfolders[index]
    top_gen(itp_files,subfolders,index,10)
    !gmx insert-molecules -f $a/newall.gro -ci itp/tip3p.gro -nmol 3400 -o $a/mof_sol.gro   
    !gmx insert-molecules -f $a/mof_sol.gro -ci itp/K.gro -nmol 10 -o $a/em.gro  
    !gmx grompp -f mdp/em.mdp   -c $a/em.gro    -p $a/em.top   -o $a/em.tpr -maxwarn 1 

In [ ]:
#neg-2 -6TCP
for index in n2_box:
    a = subfolders[index]
    top_gen(itp_files,subfolders,index,20)
    !gmx insert-molecules -f $a/newall.gro -ci itp/tip3p.gro -nmol 3400 -o $a/mof_sol.gro   
    !gmx insert-molecules -f $a/mof_sol.gro -ci itp/K.gro -nmol 20 -o $a/em.gro  
    !gmx grompp -f mdp/em.mdp   -c $a/em.gro    -p $a/em.top   -o $a/em.tpr -maxwarn 1 